# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.43.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.The dataset being used in this project is a dataset containing details of diabetes patients with many features that have been identified as indicators of diabetes. This is a classification project with the goal of using AutoML to be able to predict if someone has a possibility of having diabetes.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
import azureml.core
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'diabetes-automl'
project_folder = './training'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-204721
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-204721


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')
    
# Split the dataset into training and validation subsets
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Dataset already registered.
Data ready!


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5
}

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [6]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# TODO: Submit your experiment

automl_experiment = Experiment(ws, 'diabetes-automl')
automl_run = automl_experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-automl,AutoML_d3f8b2e3-e05a-4372-809d-2de07a085b5e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-automl,AutoML_d3f8b2e3-e05a-4372-809d-2de07a085b5e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***********************************************************************

{'runId': 'AutoML_d3f8b2e3-e05a-4372-809d-2de07a085b5e',
 'target': 'your-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-20T16:57:29.51599Z',
 'endTimeUtc': '2022-08-20T17:06:41.933987Z',
 'services': {},
 'properties': {'num_iterations': '4',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'your-compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-204721","workspace_name":"quick-starts-ws-204721","region":"southcentralus","compute_target":"your-compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:

best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.44.0, current version:1.43.0
Package:azureml-core, training version:1.44.0, current version:1.43.0
Package:azureml-dataprep, training version:4.2.2, current version:4.0.4
Package:azureml-dataprep-rslex, training version:2.8.1, current version:2.6.3
Package:azureml-dataset-runtime, training version:1.44.0, current version:1.43.0.post2
Package:azureml-defaults, training version:1.44.0, current version:1.43.0
Package:azureml-inference-server-http, training version:0.7.4, current version:0.4.13
Package:azureml-interpret, training version:1.44.0, current version:1.43.0
Package:azureml-mlflow, training version:1.44.0, current version:1.43.0.post1
Package:azureml-pipeline-core, training version:1.44.0, current version:1.43.0
Package:azureml-responsibleai, training version:1.44.0, current version:1.43.0
Package:azureml-telemetry, training version:1.44.0, current version:1.43.0
Package:azureml-train-automl-client, training version:1.44.0, curre

Run(Experiment: diabetes-automl,
Id: AutoML_d3f8b2e3-e05a-4372-809d-2de07a085b5e_2,
Type: azureml.scriptrun,
Status: Completed)

Best Model Definition:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostclassifier', XGBoostClassifier(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclass

In [9]:
from azureml.core import Model
#TODO: Save the best model


# Register model
myModel =best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_cap_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})


# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')                        

diabetes_cap_model version: 1
	 Training context : Auto ML
	 AUC : 0.9912387902233164
	 Accuracy : 0.955872594558726




## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
model = ws.models['diabetes_cap_model']
print(model.name, 'version', model.version)


diabetes_cap_model version 1


In [11]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [12]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "dservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-20 17:07:22+00:00 Creating Container Registry if not exists..
2022-08-20 17:17:23+00:00 Registering the environment..
2022-08-20 17:17:24+00:00 Use the existing image.
2022-08-20 17:17:25+00:00 Submitting deployment to compute.
2022-08-20 17:17:28+00:00 Checking the status of deployment dservice..
2022-08-20 17:22:07+00:00 Checking the status of inference endpoint dservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://e8804f02-c94f-4c5d-842d-ca6dd91bdf29.southcentralus.azurecontainer.io/score
http://e8804f02-c94f-4c5d-842d-ca6dd91bdf29.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
endpoint = 'http://a1759612-06f7-495b-8d28-175608452efd.southcentralus.azurecontainer.io/score'
key = 'TnMjFn1ohCLcmyx3BSqop6E34Enp3ZHN' 

import json
import requests

#Features for a patient
data = {
  "Inputs": {
    "data": [
      {"PatientID": 1,
      "Pregnancies": 5,
      "PlasmaGlucose": 181.0,
      "DiastolicBloodPressure": 90.6,
      "TricepsThickness": 34.0,
      "SerumInsulin": 23.0,
      "BMI": 43.51,
      "DiabetesPedigree": 1.21,
      "Age": 21.0
      }
    ]
  }
}

#Create a "data" JSON object
body = str.encode(json.dumps(data))

#Set the content type and authentication for the request
headers = {"Content-Type":"application/json",
           "Authorization":"Bearer " + key}

#Send the request
response = requests.post(endpoint, body, headers=headers)

#If we got a valid response, display the predictions
if response.status_code == 200:
    y = response.json()
    #Get the first prediction in the results
    if y["Results"][0] == 1:
        print('Diabetic')
    else:
        print("Not Diabetic")
else:
    print(response)

Diabetic


TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
print(service.get_logs())

2022-08-20T17:21:58,317284300+00:00 - rsyslog/run 
2022-08-20T17:21:58,333459700+00:00 - iot-server/run 
2022-08-20T17:21:58,335404800+00:00 - nginx/run 
2022-08-20T17:21:58,335055400+00:00 - gunicorn/run 
2022-08-20T17:21:58,346348500+00:00 | gunicorn/run | 
2022-08-20T17:21:58,379922300+00:00 | gunicorn/run | ###############################################
2022-08-20T17:21:58,390185800+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-08-20T17:21:58,400626200+00:00 | gunicorn/run | ###############################################
2022-08-20T17:21:58,401961700+00:00 | gunicorn/run | 
2022-08-20T17:21:58,412917300+00:00 | gunicorn/run | 
2022-08-20T17:21:58,438306500+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220729.v6
2022-08-20T17:21:58,447158900+00:00 | gunicorn/run | 
2022-08-20T17:21:58,455086700+00:00 | gunicorn/run | 
2022-08-20T17:21:58,460956700+00:00 | gunicorn/run | PATH environment variable: /azureml-env

In [16]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
